#### Extracting big data from the King County Assessor regarding home sales prices 
    - Block group
    - Sale price 
    - Date of sale 
    - Year of sale

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Path will varie within different directory 
df = pd.read_csv('data/EXTR_RPSale.csv', low_memory=False)

In [3]:
df.head(5)

,ExciseTaxNbr,Major,Minor,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,...,PropertyType,PrincipalUse,SaleInstrument,AFForestLand,AFCurrentUseLand,AFNonProfitUse,AFHistoricProperty,SaleReason,PropertyClass,SaleWarning
0,2687551,138860,0110,08/21/2014,245000,20140828001436,,,,,...,3,6,3,N,N,N,N,1,8,
1,1235111,664885,0040,07/09/1991,0,199203161090,071,001,664885,C,...,3,0,26,N,N,N,N,18,3,11
2,2704079,423943,0050,10/11/2014,0,20141205000558,,,,,...,3,6,15,N,N,N,N,18,8,18 31 51
3,2584094,403700,0715,01/04/2013,0,20130110000910,,,,,...,3,6,15,N,N,N,N,11,8,18 31 38
4,1056831,951120,0900,04/20/1989,85000,198904260448,117,053,951120,P,...,3,0,2,N,N,N,N,1,9,49


In [4]:
# Filtering for Major, Minor, SalePrice, DocumentDate, PropertyType
df = df.filter(items=['Major', 'Minor', 'SalePrice','DocumentDate', 'PropertyType' ])
df.head(5)

,Major,Minor,SalePrice,DocumentDate,PropertyType
0,138860,0110,245000,08/21/2014,3
1,664885,0040,0,07/09/1991,3
2,423943,0050,0,10/11/2014,3
3,403700,0715,0,01/04/2013,3
4,951120,0900,85000,04/20/1989,3


In [18]:
# Filtering for sale price greater than $0
df_filtered = df.query('SalePrice>=1')
df_filtered.head(5)

,Major,Minor,SalePrice,DocumentDate,PropertyType
0,138860,0110,245000,08/21/2014,3
4,951120,0900,85000,04/20/1989,3
6,022209,9001,5755000,04/25/2001,1
7,022209,9003,5755000,04/25/2001,1
8,022209,9007,5755000,04/25/2001,1


In [22]:
# Filtering for single family home sales (Property type  = 11)
filteredData = df_filtered.query('PropertyType == 11')

In [23]:
filteredData.head(5)

,Major,Minor,SalePrice,DocumentDate,PropertyType
313,221250,0090,400000,04/12/2017,11
339,701120,1162,725000,11/02/2016,11
395,732190,0030,1000000,01/25/2018,11
580,799960,0065,5000,04/13/2011,11
679,277060,6955,607500,07/06/2017,11


In [26]:
# Joining Major + Minor to create ParcelID
ParcelId = filteredData.apply(lambda x:'%s-%s' % (x['Major'],x['Minor']),axis=1)
ParcelID = pd.DataFrame(data=ParcelId, columns=['PercelID'])
ParcelID.head(5)

,PercelID
313,221250-0090
339,701120-1162
395,732190-0030
580,799960-0065
679,277060-6955


In [35]:
KCSales = pd.concat([filteredData, ParcelID], sort=True)
KCSales.head(10)

,DocumentDate,Major,Minor,PercelID,PropertyType,SalePrice
313,04/12/2017,221250,0090,NaN,11.0,400000.0
339,11/02/2016,701120,1162,NaN,11.0,725000.0
395,01/25/2018,732190,0030,NaN,11.0,1000000.0
580,04/13/2011,799960,0065,NaN,11.0,5000.0
679,07/06/2017,277060,6955,NaN,11.0,607500.0
685,08/01/2016,382220,0136,NaN,11.0,405000.0
727,05/10/2017,231000,0480,NaN,11.0,410000.0
965,02/26/2018,785336,0400,NaN,11.0,850000.0
1092,05/03/2018,401380,0208,NaN,11.0,10.0
1105,04/20/2017,073220,0010,NaN,11.0,289740.0


#### Extracting Centroid using pysal to extract the geographic center points of the coordinates

In [4]:
import pysal as ps 
import xml.etree.ElementTree as ET
import urllib.request

In [5]:
# Path will change within different directory 
inputfile = "data\parcel\parcel.shp"

In [11]:
# Reading shapefile via pysal 
df_Shapefile = ps.pdio.read_files(inputfile)
df_Shapefile.head(5)

,MAJOR,MINOR,PIN,Shape_area,Shape_len,geometry
0,000020,0001,0000200001,367523.511641,4889.923177,<pysal.cg.shapes.Polygon object at 0x000000001...
1,000020,0003,0000200003,5446.331001,330.703378,<pysal.cg.shapes.Polygon object at 0x000000001...
2,000020,0006,0000200006,6000.064502,340.002037,<pysal.cg.shapes.Polygon object at 0x000000001...
3,000020,0007,0000200007,7200.080077,360.002087,<pysal.cg.shapes.Polygon object at 0x000000001...
4,000020,0008,0000200008,6068.041700,317.359849,<pysal.cg.shapes.Polygon object at 0x000000001...


In [12]:
# Convert to Lat/Long (from state plane coordinates)
def XYtoLatLong(X, Y):
    URL = "http://citygis/coordinateconversion/coordinateconversion.asmx/WaStatePlaneToLatLong?X=" + str(X) + "&Y=" + str(Y)
    tree = ET.parse(urllib.request.urlopen(URL))
    doc = tree.getroot()
    return (float(doc[0].text), float(doc[1].text))


In [ ]:
# Iterate over each parcel to get Lat/Long
Latitude = []
Longitude = []

for index, row in df_Shapefile.iterrows():   
        X = row['geometry'].centroid[0]
        Y = row['geometry'].centroid[1]
        Coordinates = XYtoLatLong(X,Y)
        Latitude.append(Coordinates[0])
        Longitude.append(Coordinates[1])

df_Shapefile['lat'] = Latitude
df_Shapefile['lon'] = Longitude

In [ ]:
#df_shapefile.head(5)

In [11]:
df_kcsales = pd.read_csv('KCSales.csv')

In [12]:
df_kcsales.head(-5)

,Unnamed: 0,Major,Minor,SalePrice,DocumentDate,PropertyType,ParcelID,lat,lon
0,0,221250,90,400000,04/12/2017,11,2212500090,47.33645,-122.19687
1,1,701120,1162,725000,11/02/2016,11,7011201162,47.63718,-122.36817
2,2,732190,30,1000000,01/25/2018,11,7321900030,NaN,NaN
3,3,799960,65,5000,04/13/2011,11,7999600065,47.47379,-122.26811
4,4,277060,6955,607500,07/06/2017,11,2770606955,47.65817,-122.39068
5,5,382220,136,405000,08/01/2016,11,3822200136,47.72822,-122.29669
6,6,231000,480,410000,05/10/2017,11,2310000480,47.35566,-122.03910
7,7,785336,400,850000,02/26/2018,11,7853360400,47.51670,-121.87502
8,8,401380,208,10,05/03/2018,11,4013800208,47.32307,-122.28195
9,9,73220,10,289740,04/20/2017,11,732200010,47.69011,-122.15724


In [7]:
# Filtering for sale price greater than $0
df_kcsales = df_kcsales.query('SalePrice>=100000')

In [18]:
df_kcsales = df_kcsales.dropna()
#df_kcsales.head(-5)

In [19]:
df_kcsales = df_kcsales.drop(['Major', 'Minor' ], axis=1)

In [20]:
df_kcsales.head(5)

,Unnamed: 0,SalePrice,DocumentDate,PropertyType,ParcelID,lat,lon
0,0,400000,04/12/2017,11,2212500090,47.33645,-122.19687
1,1,725000,11/02/2016,11,7011201162,47.63718,-122.36817
3,3,5000,04/13/2011,11,7999600065,47.47379,-122.26811
4,4,607500,07/06/2017,11,2770606955,47.65817,-122.39068
5,5,405000,08/01/2016,11,3822200136,47.72822,-122.29669


In [21]:
df_kcsales = df_kcsales[['ParcelID', 'DocumentDate', 'PropertyType', 'SalePrice', 'lat', 'lon']]

In [22]:
df_kcsales.head(5)

,ParcelID,DocumentDate,PropertyType,SalePrice,lat,lon
0,2212500090,04/12/2017,11,400000,47.33645,-122.19687
1,7011201162,11/02/2016,11,725000,47.63718,-122.36817
3,7999600065,04/13/2011,11,5000,47.47379,-122.26811
4,2770606955,07/06/2017,11,607500,47.65817,-122.39068
5,3822200136,08/01/2016,11,405000,47.72822,-122.29669


In [10]:
df_kcsales_blkgrps.to_csv('KCSales.csv', index=False)